In [19]:
import pandas as pd
import requests
import pymysql.cursors
import pyspark
import json

In [12]:
class Config:
  MYSQL_HOST = 
  MYSQL_PORT = 3306  # default port ของ MySQL คือ 3306
  MYSQL_USER = 
  MYSQL_PASSWORD = 
  MYSQL_DB = 
  MYSQL_CHARSET = 

In [13]:
connection = pymysql.connect(host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor)

In [14]:
cursor = connection.cursor()
cursor.execute("show tables;")
tables = cursor.fetchall()
cursor.close()
print(tables)

[{'Tables_in_detraining': 'current_dept_emp'}, {'Tables_in_detraining': 'departments'}, {'Tables_in_detraining': 'dept_emp'}, {'Tables_in_detraining': 'dept_emp_latest_date'}, {'Tables_in_detraining': 'dept_manager'}, {'Tables_in_detraining': 'employees'}, {'Tables_in_detraining': 'online_retail'}, {'Tables_in_detraining': 'salaries'}, {'Tables_in_detraining': 'titles'}]


In [15]:
with connection.cursor() as cursor:
  # Read a single record
  sql = "SELECT * FROM online_retail"
  cursor.execute(sql)
  result = cursor.fetchall()

print("number of rows: ", len(result))

number of rows:  161331


In [16]:
df = pd.DataFrame(result)

In [25]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2018 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2018 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
161326,550638,22960,JAM MAKING SET WITH JARS,1,19/4/2019 15:44,8.29,NaN,United Kingdom
161327,550638,22961,JAM MAKING SET PRINTED,1,19/4/2019 15:44,1.45,NaN,United Kingdom
161328,550638,22978,PANTRY ROLLING PIN,1,19/4/2019 15:44,7.46,NaN,United Kingdom
161329,550638,22982,PANTRY PASTRY BRUSH,1,19/4/2019 15:44,2.46,NaN,United Kingdom


In [26]:
df.describe()

,InvoiceNo,Quantity,UnitPrice,CustomerID
count,161331.000000,161331.000000,161331.000000,111388.000000
mean,543335.175688,9.936392,4.152132,15293.393471
std,4128.304528,191.110632,38.918686,1732.404778
min,536365.000000,-5368.000000,0.000000,12346.000000
25%,539750.000000,1.000000,1.250000,13851.000000
50%,543191.000000,3.000000,2.250000,15194.000000
75%,546892.000000,10.000000,4.210000,16873.000000
max,550638.000000,74215.000000,13541.330000,18283.000000


In [18]:
url = ""
response = requests.get(url)

In [20]:
result = response.json()

In [21]:
result_conversion_rate = json.loads(response.text)

In [22]:
conversion_rate = pd.DataFrame.from_dict(result_conversion_rate)

In [23]:
conversion_rate.head()

,Rate
2018-01-01T00:00:00.000Z,43.9991
2018-01-02T00:00:00.000Z,44.0731
2018-01-03T00:00:00.000Z,43.7025
2018-01-04T00:00:00.000Z,43.6914
2018-01-05T00:00:00.000Z,43.6490


In [24]:
conversion_rate = conversion_rate.reset_index().rename(columns={"index":"date"})
conversion_rate.head()

,date,Rate
0,2018-01-01T00:00:00.000Z,43.9991
1,2018-01-02T00:00:00.000Z,44.0731
2,2018-01-03T00:00:00.000Z,43.7025
3,2018-01-04T00:00:00.000Z,43.6914
4,2018-01-05T00:00:00.000Z,43.6490


In [29]:
df['InvoiceTimestamp'] = df['InvoiceDate']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2018 08:26,2.55,17850.0,United Kingdom,1/12/2018 08:26
1,536365,71053,WHITE METAL LANTERN,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2018 08:26,2.75,17850.0,United Kingdom,1/12/2018 08:26
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26


In [30]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date

In [32]:
final_df = df.merge(conversion_rate, how="left", left_on="InvoiceDate", right_on="date")
final_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp,date,Rate
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2018-01-12,2.55,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
1,536365,71053,WHITE METAL LANTERN,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2018-01-12,2.75,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
...,...,...,...,...,...,...,...,...,...,...,...
161326,550638,22960,JAM MAKING SET WITH JARS,1,2019-04-19,8.29,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622
161327,550638,22961,JAM MAKING SET PRINTED,1,2019-04-19,1.45,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622
161328,550638,22978,PANTRY ROLLING PIN,1,2019-04-19,7.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622
161329,550638,22982,PANTRY PASTRY BRUSH,1,2019-04-19,2.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622


In [33]:
final_df['THBPrice'] = final_df['UnitPrice'].multiply(final_df['Rate'])

In [34]:
final_df.to_csv('final_df',index= False)